# Author Name Recon

This notebook cleans the author names of each paper for the top 5 economic journals for all time using jstor metadata. Running through the cells produces a reconciliation file for the author names as they appear in the masterlists in a raw format. ie: what went in for each author name of an article, what came out and possible aliases. I exclude Miscellaneous content only.

## Notes

* Going through this data, some of the citation files for JPE acquired from the chicago journals website has an error where special characters are not encoding properly. I have created a manual resolution of these files, but this should be fixed further up the pipeline. There are only 33 names with this problem and you can't resolve it by using the unidecode library. But, specifically, these files:

{'uchicago_jpe126_1.bib',
 'uchicago_jpe126_3.bib',
 'uchicago_jpe126_5.bib',
 'uchicago_jpe126_6.bib',
 'uchicago_jpe126_S1.bib',
 'uchicago_jpe127_1.bib',
 'uchicago_jpe127_2.bib',
 'uchicago_jpe127_3.bib',
 'uchicago_jpe127_4.bib',
 'uchicago_jpe127_5.bib',
 'uchicago_jpe128_1.bib',
 'uchicago_jpe128_10.bib',
 'uchicago_jpe128_11.bib',
 'uchicago_jpe128_12.bib',
 'uchicago_jpe128_2.bib',
 'uchicago_jpe128_4.bib',
 'uchicago_jpe128_5.bib',
 'uchicago_jpe128_7.bib',
 'uchicago_jpe128_8.bib',
 'uchicago_jpe128_9.bib'} 
* Resolution of some special characters results in empty string using unicodedata.normalize() function. For example the danish o (o with a slash) does not result in o but rather an empty string. So I have made a function and a accompanying recon file that fixes this.
* There are some authors which:
    * have only their initials (even the last name is contracted)
    * they are just referred to by their last name because of how well known they are. 
    * there are so many authors that only the primary author is mentioned and the others are contracted into a catch-all term like "other", "company" or "co.". 
    
    These are limited and recommend resolving here as they are easily picked out. Todo: Construct a recon file manually compiled to replace these.

**Input**: 
* each of the cleaned masterlists with file ending in \_M_sco_du.xlsx 

**Output**: 
* a json file of reconciliated and cleaned author names and associated article URLs
* a list of UTF-8 characters and the ascii equivalent to which they should resolve.
* a list of author name corrections for JPE errors
* TODO: a list of author name corrections for contraction cases, need to do article URL matching.

## Import libraries

In [243]:
import pandas as pd
from unidecode import unidecode
import re
from datetime import date
import json
import numpy as np
import string
import time
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Read in input files

In [244]:
JOURNALS= ['AER', 'JPE', 'ECTA', 'RES', 'QJE']
base_path="/Users/sijiawu/Work/Thesis/Data/"
#read in all processed masterlists


j_data=pd.read_pickle(base_path+"Combined/011_merged_proc_scopus_inception_2020.pkl")
j_data['year']=j_data["year"].astype(int)

In [245]:
j_data.columns # verify headers

Index(['issue_url', 'author', 'title', 'journal', 'volume', 'number', 'pages',
       'year', 'ISSN', 'abstract', 'URL', 'publisher', 'content_type', 'type',
       'jid', 'author_split', 'urldate', 'reviewed-author', 'uploaded',
       'title_10', 'URL_og', 'number_og', 'title_og', 'author_og', 'pages_og',
       'j_fix', 'scopus_jid', 'scopus_id', 'scopus_authorgroup',
       'scopus_authors', 'scopus_affiliations', 'scopus_references',
       'scopus_author_full_names', 'scopus_title', 'scopus_year',
       'scopus_source_title', 'scopus_volume', 'scopus_issue', 'scopus_art_no',
       'scopus_page_start', 'scopus_page_end', 'scopus_page_count',
       'scopus_cited_by', 'scopus_doi', 'scopus_abstract', 'scopus_publisher',
       'scopus_document_type', 'scopus_publication_stage',
       'scopus_open_access', 'scopus_source', 'scopus_eid', 'scopus_title_og',
       'scopus_volume_og', 'scopus_issue_og', 'scopus_page_start_og',
       'scopus_page_end_og', 'scopus_year_og', 's_fix', 

In [246]:
len(j_data["URL"].unique()) #number of articles

62257

In [247]:
j_data[j_data['URL']=="https://www.jstor.org/stable/27804963"].values

array([['https://www.jstor.org/stable/10.2307/i27804949',
        'Victoria Ivashina and David Scharfstein',
        'loan syndication and credit cycles',
        'The American Economic Review', 100, '2', '57-61', 2010, 28282,
        nan, 'https://www.jstor.org/stable/27804963',
        'American Economic Association', 'Article', 'S', 'aer', nan,
        datetime.datetime(2023, 9, 4, 0, 0), nan, 1.0,
        'Loan Syndication and Credit Cycles',
        'https://www.jstor.org/stable/27804963', '2',
        'Loan Syndication and Credit Cycles', nan, '57-61', 1.0, 'aer',
        '10.1257/aer.100.2.57', True, True, True, True,
        'Ivashina, Victoria (26323576100); Scharfstein, David (7003408304)',
        'loan syndication and credit cycles', '2010',
        'American Economic Review', '100', '2', nan, '57', '61', 4.0,
        65.0, '10.1257/aer.100.2.57', '[No abstract available]', nan,
        'Conference paper', 'Final', nan, 'Scopus', '2-s2.0-77956122893',
        'Loan syndicat

In [248]:
# must account for duplicates due to scopus duplication - scopus has one jstor to many scopus entries
j_data[['content_type', 'URL']].drop_duplicates()['content_type'].value_counts()

content_type
Article       32691
Review        13052
MISC          12530
Comment        1419
Reply           832
Review2         761
Discussion      742
Rejoinder       153
Errata           77
Name: count, dtype: int64

## Create a restricted character set

In [249]:
char_set="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ,'.-"
chars=[*char_set]

## Constants

In [250]:
# titles that do not matter
auth_ad=[
    "c.p.a.",
    "m.e.",
    "m.d.",
    "s.j.", #society of jesus
    "s. j.", #society of jesus
    # "wm.",  #contraction of the given name William
]

# suffixes that matter for distinguishing authors
auth_ad2=[
    "2nd",
    "3rd",
    "jr.", #junior
    "yr." #misspelling of junior. Yordon case
]

## Resolve the JPE formatting issue

Please generate the file using the code block at the end of the notebook and manually resolve the errors. Rename the file and run the next two codeblocks.

In [251]:
with open('jpe_problem_names.json') as f: 
    data = f.read() 
names_repl = json.loads(data) 

In [252]:
for i in j_data.index:
    if j_data.loc[i, "author"] in names_repl.keys():
         j_data.loc[i, "author"]=names_repl[j_data.loc[i, "author"]]

## Resolve special characters outside ascii

As above, generate the file using code at the end of the notebook. Or copy in the file.

In [253]:
with open('spec_char_replacement.json') as f: 
    data = f.read() 
js = json.loads(data) 

def replace_spec_chars(str_in):
    hold=""
    for o in str_in:
        if o.lower() in js.keys():
            hold=hold+js[o.lower()]
        else:
            hold=hold+o
    return hold

In [254]:
for i in j_data.index:
    if pd.isna(j_data.loc[i, "author"])==False:
        j_data.loc[i,'author']=re.sub('\xa0'," ",j_data.loc[i,'author'])
        j_data.loc[i,'author']=re.sub(r'\s+', " ", j_data.loc[i,'author'])
        if j_data.loc[i, "author"].isascii()==True:
            continue
        j_data.loc[i, "author"]=replace_spec_chars(j_data.loc[i,"author"])
        if j_data.loc[i, "author"].isascii()==False:
            print(j_data.loc[i, "author"])

Mariagiovanna Baccara and Ayse İmrohoroglu and Alistair J. Wilson and Leeat Yariv
Kaiji Chen and Ayse İmrohoroglu and Selahattin İmrohoroglu
Ayse İmrohoroglu and Selahattin İmrohoroglu and Douglas H. Joines
Luisa Fuster and Ayse İmrohoroglu and Selahattin İmrohoroglu
Ayse İmrohoroglu and Selahattin İmrohoroglu and Douglas H. Joines


Create a new column where all author names are split by " and ".

In [255]:
j_data['author_split']=j_data['author'].str.split(" and ")
j_data['author_count']=j_data['author_split'].apply(lambda x: len(x) if isinstance(x, list) else 0)

## Process author names

In [256]:
#store the aliases in lists for comparison later
a1=[]
a2=[]
i1=[]
ln=[]


#function for creating aliases
#todo annotate function
def proc_auth(auth_str):
#     print(auth_str)
    suff=""
    auth_split=auth_str.split(" ")
    if "," in auth_str:
        suff=", "+auth_str.split(", ")[-1]
        auth_split=auth_str.split(",")[0].split(" ")
        # print(suff)
    check=0
    for j in auth_split:
        if "." in j:
            check=check+1
    if check==len(auth_split):
        return [auth_str, auth_str, auth_str, auth_str, 0]
    
    if len(auth_split)>1:
        init_auth=auth_split[0][0]+'. '+auth_split[-1]
        alt_auth=""
        alt_2_auth=auth_split[0]+" "
        if len(auth_split)>2:
            for k in auth_split[1:-1]:
                alt_2_auth+=k[0]+'. '
        alt_2_auth+=auth_split[-1]
        
        
        for k in auth_split[:-1]:
            alt_auth+=k[0]+'. '
        alt_auth+=auth_split[-1]
        ln.append(auth_split[-1])
        a1.append(alt_2_auth+suff)
        a2.append(alt_auth+suff)
        i1.append(init_auth+suff)
        
        
        
        return [alt_2_auth, alt_auth, init_auth, auth_split[-1]]
    else:
         return[auth_str, auth_str, auth_str, auth_str, 0]


In [257]:
for i in j_data.index:
    authors=j_data.loc[i,"author_split"]

In [258]:
j_data['jid'].unique()

array(['qje', 'aer', 'ecta', 'jpe', 'res'], dtype=object)

Note that there was a bug in the code below that has not resolved the case of authors having a designation like m.e. at the end of their name. This is resolved.

In [259]:
a=1
all_authors=[]
all_authors_a=[]
autht=0
proc_auths_all={}
for i in j_data.index:
    authors=j_data.loc[i,"author_split"]
    proc_auths={
                "authors":{}, 
                "year":j_data.loc[i,"year"], 
                'content_type':j_data.loc[i, "content_type"],
                'jid':j_data.loc[i, "jid"]
                }
    if pd.isna(j_data.loc[i,"author"])==False:
        for j in range(len(authors)):
            sa=authors[j].lower()
            all_authors_a.append(str(sa))
            autht=autht+1

            o=[]
            
            # remove the author titles that are not necessary if present
            for k in auth_ad:
                p=", "+k
                if (p in sa):
                    if 'yr' in p:
                        print(sa)
                    sa=re.sub(p,'',sa)
                    # print(sa)
                    break

            # remove and store the suffixes of an author if present
            for k in auth_ad2:
                # print("case 2")

                r=", "+k
                if r in sa[-len(r):]:
                    # print("case 2.1")
                    o.append(k)
                    sa=re.sub(r,'',sa)
                    # print(sa)
                    break
                elif " "+k in sa[-len(" "+k):]:
                    # print("case 2.2")
                    t1=sa.split(" "+k)
                    sa=re.sub(" "+k,"",sa)
                    # print(sa)
                    o.append(k)
                    break
                    
                        
            if "," in sa:
                reorg=sa.split(r", ")
                sa=reorg[1].strip()+ " "+ reorg[0].strip()
           
            all_authors.append(sa)
            proc_auths["authors"][j]={}
            proc_auths["authors"][j]["raw"]=authors[j]
            proc_auths["authors"][j]["init"]=sa
            proc_auths["authors"][j]['auth_suffix']=o
            aliases=proc_auth(sa)
            if len(aliases)==5:
                k=1
                # print(sa)
                # print(j_data.loc[i,"author"])
                # print(j_data.loc[i, "URL"])
                # print(j_data.loc[i, "year"])
                # print(j_data.loc[i, "journal"])

            proc_auths["authors"][j]["a1"]=aliases[0]
            proc_auths["authors"][j]["a2"]=aliases[1]
            proc_auths["authors"][j]["a3"]=aliases[2]

            if ("[" in sa) or ("(" in sa):
                k=1
                # print(sa)
                # print(j_data.loc[i,"author"])
                # print(j_data.loc[i, "year"])
                # print(j_data.loc[i, "journal"])
                # print(j_data.loc[i, "URL"])

    else:
        a=a+1
    proc_auths_all[j_data.loc[i,"URL"]]=proc_auths

## Save the output

In [260]:

with open("author_proc.json", "w") as outfile: 
    json.dump(proc_auths_all, outfile, indent=4, default=int)

j_data.to_pickle(base_path+"Combined/020_merged_proc_scopus_inception_with_auth_split_2020.pkl")


## Compute some stats

In [261]:
auth_summary=[]
auth_t=list(set(all_authors_a))
u_auth=list(set(all_authors))
u_a1=list(set(a1))
u_a2=list(set(a2))
u_a3=list(set(i1))
u_ln=list(set(ln))
auth_summary.append({"description":"The total number of authors", "total": len(all_authors)})
auth_summary.append({"description":"The number of unique author names without editing", "total":  len(auth_t)})
auth_summary.append({"description":"The number of unique author names after initial processing", "total":  len(u_auth)})
auth_summary.append({"description":"Unique alias 1 type names (contracted middle names + last name)", "total":  len(u_a1)})
auth_summary.append({"description":"Unique alias 2 type names (contracted first names + last name)", "total":  len(u_a2)})
auth_summary.append({"description":"Unique alias 3 type names (contracted first name + last name)", "total":  len(u_a3)})
auth_summary.append({"description":"Unique last names", "total":  len(u_ln)})

pd.DataFrame(auth_summary).to_csv("summary_author_counts.csv", index=False)

## Computing similarity to find duplicates
Methodology with 20000 names, we can precommpile the set of potential names using last names and first names as a way to determine duplicates. 
1. allocate authors to a dictionary by last names.
2. Function by process of elimination. Assumption: first letter of last name and first letter of first name will always be correct.
3. format 

In [262]:
a1_dict={}
a2_dict={}
a3_dict={}
similar={}

for i in string.ascii_lowercase:
    a1_dict[i]={}
    a2_dict[i]={}
    a3_dict[i]={}
    similar[i]={}
    for j in string.ascii_lowercase:
        a1_dict[i][j]={}
        a2_dict[i][j]={}
        a3_dict[i][j]=[]
        similar[i][j]=[]

for i in u_a1:
    init_split=i.split(',')
    suff=init_split[-1]
    auth_split=init_split[0].split(" ")
    l=auth_split[-1][0]
    f=auth_split[0][0] 
    lname=auth_split[-1]
    try:
        if lname in a1_dict[l][f].keys():
            a1_dict[l][f][lname].append(i)
        else:
            a1_dict[l][f][lname]=[i]
    except:
        print(i)

for i in u_a3:
    init_split=i.split(',')
    suff=init_split[-1]
    auth_split=init_split[0].split(" ")
    l=auth_split[-1][0]
    f=auth_split[0][0] 
    try:
        a3_dict[l][f].append(i)
    except:
        print(i)


(suggested b. f. r. velde)
(suggested b. a. khanna)
(suggested b. r. e. cumby
staff o. s. s. 125
(. cumby
(. khanna)
s. 125
(. velde)


In [263]:
j_data['content_type'].unique()

array(['Article', 'MISC', 'Comment', 'Reply', 'Errata', 'Rejoinder',
       'Discussion', 'Review', 'Review2'], dtype=object)

In [264]:
max=0
max3=0

for i in string.ascii_lowercase:
    for j in string.ascii_lowercase:
        print(i + " " +j+ ": "+str(len(a1_dict[i][j])))

        a3_dict[i][j]=list(set(a3_dict[i][j]))
        a3_dict[i][j].sort()
        print(i + " " +j+ ": "+str(len(a3_dict[i][j])))
        
        
        if len(a1_dict[i][j])>max:
            max=len(a1_dict[i][j])
        if len(a3_dict[i][j])>max3:
            max3=len(a3_dict[i][j])

print(max)
print(max3)

a a: 71
a a: 71
a b: 15
a b: 15
a c: 31
a c: 31
a d: 39
a d: 39
a e: 34
a e: 34
a f: 19
a f: 19
a g: 26
a g: 26
a h: 31
a h: 31
a i: 10
a i: 10
a j: 61
a j: 61
a k: 17
a k: 17
a l: 28
a l: 28
a m: 50
a m: 50
a n: 13
a n: 13
a o: 10
a o: 10
a p: 26
a p: 26
a q: 1
a q: 1
a r: 53
a r: 53
a s: 48
a s: 48
a t: 18
a t: 18
a u: 1
a u: 1
a v: 10
a v: 10
a w: 23
a w: 23
a x: 0
a x: 0
a y: 7
a y: 7
a z: 2
a z: 2
b a: 141
b a: 141
b b: 64
b b: 64
b c: 80
b c: 80
b d: 90
b d: 90
b e: 88
b e: 88
b f: 48
b f: 48
b g: 81
b g: 81
b h: 78
b h: 78
b i: 19
b i: 19
b j: 165
b j: 165
b k: 52
b k: 52
b l: 67
b l: 67
b m: 126
b m: 126
b n: 37
b n: 37
b o: 17
b o: 17
b p: 66
b p: 66
b q: 0
b q: 0
b r: 141
b r: 141
b s: 97
b s: 97
b t: 56
b t: 56
b u: 2
b u: 2
b v: 22
b v: 22
b w: 70
b w: 70
b x: 0
b x: 0
b y: 14
b y: 14
b z: 4
b z: 4
c a: 93
c a: 93
c b: 39
c b: 39
c c: 69
c c: 69
c d: 81
c d: 81
c e: 57
c e: 57
c f: 44
c f: 44
c g: 57
c g: 57
c h: 37
c h: 37
c i: 8
c i: 8
c j: 144
c j: 144
c k: 37
c k: 37
c 

In [265]:
for i in a1_dict["a"]["b"].keys():
    print(a1_dict["a"]["b"][i])

['bee y. aw']
['beth allen', 'bruce t. allen', 'benjamin j. allen']
['bevin ashenmiller']
['brant abbott']
['brian j. aitken']
['b. p. adarkar']
['bruce n. angier']
['bushrod w. allin']
['b. k. atrostic']
['bernard e. anderson', 'b. m. anderson', 'benjamin m. anderson']
['bernardino adao']
['burton a. abrams']
['bo axell']
['bijan b. aghevli']
['belinda archibong']


In [266]:

for m in string.ascii_lowercase:
    for j in string.ascii_lowercase:
        temp=[]
        source=a1_dict[m][j].keys()        
        for i in source: 
            sims=process.extract(i, source, 
                                 limit=len(source), 
                                 #scorer=fuzz.partial_ratio
                                ) 
            sims_set=[]
            for k in sims:
                if k[1]>90:
                    sims_set.append(k[0])
            if len(sims_set)>1:
                # print(sims_set)
                temp.append(tuple(sims_set))
        
        similar[m][j]=list(set(map(tuple, map(sorted, temp))))


In [267]:
# for i in string.ascii_lowercase:
#     similar[i]=list(set(map(tuple, map(sorted, similar[i]))))

In [268]:
for m in string.ascii_lowercase:
    for l in string.ascii_lowercase:
        print(m+ ' '+l)
        print(similar[m][l])
        for i in similar[m][l]:
            print()
            temp=[]
            for j in list(i):
                temp+=a1_dict[m][l][j]
            sims=process.extract(temp[0], temp, 
                                         limit=len(temp), 
                                         scorer=fuzz.partial_ratio
                                        ) 
            print(temp)
            
            sims_set=[]
            for k in sims:
                print(k)
                if k[1]>80:
                    sims_set.append(k[0])
            print(sims_set)

a a
[('abdulkadiroglu', 'abdulkadroglu')]

['atila abdulkadiroglu', 'atila abdulkadroglu']
('atila abdulkadiroglu', 100)
('atila abdulkadroglu', 95)
['atila abdulkadiroglu', 'atila abdulkadroglu']
a b
[]
a c
[]
a d
[('atkin', 'atkins')]

['david atkin', 'david atkins']
('david atkin', 100)
('david atkins', 100)
['david atkin', 'david atkins']
a e
[('allinson', 'allison')]

['edward p. allinson', 'elizabeth allison']
('edward p. allinson', 100)
('elizabeth allison', 53)
['edward p. allinson']
a f
[]
a g
[]
a h
[]
a i
[]
a j
[('agell', 'angell')]

['jonas agell', 'james w. angell']
('jonas agell', 100)
('james w. angell', 64)
['jonas agell']
a k
[]
a l
[]
a m
[]
a n
[]
a o
[]
a p
[]
a q
[]
a r
[('arnold', 'arnould'), ('agarwal', 'aggarwal')]

['robert k. arnold', 'richard j. arnould']
('robert k. arnold', 100)
('richard j. arnould', 62)
['robert k. arnold']

['rajshree agarwal', 'rajesh k. aggarwal']
('rajshree agarwal', 100)
('rajesh k. aggarwal', 69)
['rajshree agarwal']
a s
[('altman'

In [269]:
sims_set

['erich w. zimmerman', 'erich w. zimmermann']

In [270]:
a1_dict["e"]["g"]

{'evans': ['glendower evans',
  'griffith c. evans',
  'george w. evans',
  'g. b. a. evans',
  'george h. evans',
  'george evans',
  'g.c. evans',
  'g. h. evans'],
 'engelhardt': ['gary v. engelhardt'],
 'eddy': ['george a. eddy'],
 'eggertsson': ['gauti b. eggertsson', 'gauti eggertsson'],
 'ensley': ['grover w. ensley', 'g. w. ensley'],
 'emerson': ['guy emerson'],
 'elliott': ['g. a. elliott', 'george a. elliott', 'graham elliott'],
 'ellison': ['glenn ellison'],
 'edwards': ['geoff edwards', 'george w. edwards'],
 'erb': ['g. f. erb'],
 'ellis': ['gregory m. ellis'],
 'eads': ['george eads', 'george c. eads'],
 'egorov': ['georgy egorov'],
 'eliasson': ['gunnar eliasson'],
 'eberle': ['george j. eberle']}

# Three cases of ambiguity:

1. same name, different person
   - is in same time period -> require affiliations
   - not in same time period -> resolve with significant significant publishing year barrier
2. similar name or alias, same person
   - has multiple resolution from no middle name to has middle name: require affiliations
   - spelling error possibilities. Direct correction in data - require research on individual articles for resolution, restricted to this data set only
3. has multiple resolutions from contraction: require affiliations
   - generate list of contractions and associated possibilities if full name exists
   - modify by 


In [271]:
p=0
q=0
for a in u_a1:
    if "." in a[:2]:
        p+=1
        temp=a.split(" ")
        if temp[-1] in a1_dict[temp[-1][0]][temp[0][0]].keys():
            n_list=a1_dict[temp[-1][0]][temp[0][0]][temp[-1]]
            if len(n_list)>1:
                if len(temp)>2:
                    a_list=[]
                    for k in n_list:
                        temp2=k.split(" ")
                        if len(temp2)>2:
                            if temp2[1][0]==temp[1][0]:
                                a_list.append(k)
                        else:
                            a_list.append(k)
                    if len(a_list)>1:
                        print(a)
                        print(a_list)
                    else:
                        continue
                else:
                    print(a)
                    print(n_list)
                q+=1
                
        
print(p)
print(q)

g. b. hotchkiss
['g. b. hotchkiss', 'george b. hotchkiss']
d. j. smyth
['d. j. smyth', 'david smyth', 'd. smyth', 'david j. smyth']
e. baudier
['e. baudier', 'edmond baudier']
e. e. witte
['e. e. witte', 'edwin e. witte']
l. r. gottlieb
['l. r. gottlieb', 'louis r. gottlieb']
c. j. bullock
['c. j. bullock', 'charles j. bullock']
p. diamond
['p. diamond', 'peter a. diamond', 'peter diamond']
m. c. urquhart
['m. c. urquhart', 'malcolm c. urquhart']
g. santoni
['g. santoni', 'gary santoni']
a. holly
['a. holly', 'alberto holly']
w. e. hotchkiss
['w. e. hotchkiss', 'willard e. hotchkiss', 'william e. hotchkiss']
a. d. foville
['a. d. foville', 'alfred d. foville']
j. d. black
['j. d. black', 'j. black', 'john d. black']
p. t. homan
['p. t. homan', 'paul t. homan']
r. j. gonzalez
['r. j. gonzalez', 'robert gonzalez']
a. d. weber
['a. d. weber', 'andrea weber']
j. w. f. rowe
['j. w. f. rowe', 'j. rowe', 'john w. rowe']
i. m. biss
['i. m. biss', 'irene m. biss']
m. h. robinson
['m. h. robinso

In [272]:
for m in string.ascii_lowercase:
    for l in string.ascii_lowercase:
        print(a1_dict[m][l].keys())

dict_keys(['aladangady', 'asimakopulos', 'amano', 'atkinson', 'anderson', 'aizcorbe', 'adams', 'attar', 'atkeson', 'aughinbaugh', 'amershi', 'achinstein', 'alstadsaeter', 'ando', 'abdulkadiroglu', 'abrahamson', 'ascher', 'ashcraft', 'auerbach', 'abdulkadroglu', 'abiad', 'aradillas-lopez', 'anthony', 'acolin', 'arvanitis', 'arragon', 'abramson', 'ang', 'admati', 'andersen', 'auclert', 'abadie', 'agrawal', 'alesina', 'alkan', 'ambrus', 'amsden', 'alchian', 'adsera', 'araujo', 'atwood', 'alexander', 'abel', 'al-samarrie', 'acconcia', 'akerman', 'abraham', 'ajello', 'almazan', 'alnasrawi', 'aizer', 'agapos', 'agan', 'abulafi', 'araoz', 'attema', 'arbel', 'andrew', 'armand', 'anastasopoulos', 'abrassart', 'ayres', 'ayzenshtadt', 'arabmazar', 'akhmedov', 'ades', 'atakan', 'acharya', 'abouchar', 'arad', 'adhvaryu'])
dict_keys(['aw', 'allen', 'ashenmiller', 'abbott', 'aitken', 'adarkar', 'angier', 'allin', 'atrostic', 'anderson', 'adao', 'abrams', 'axell', 'aghevli', 'archibong'])
dict_keys(['

## Top 20 Author Output Ranked for all time

In [273]:
# after initial process
pd.DataFrame(all_authors).value_counts()[:20]

0                   
frank h. knight         113
george j. stigler        95
a. b. wolfe              92
paul a. samuelson        91
daron acemoglu           85
f. w. taussig            84
william j. baumol        83
m. bronfenbrenner        81
j. m. clark              79
jean tirole              77
joseph e. stiglitz       75
paul h. douglas          71
h. parker willis         68
wesley c. mitchell       67
milton friedman          66
franklin m. fisher       64
harry g. johnson         63
t. n. carver             60
chester w. wright        57
j. laurence laughlin     57
Name: count, dtype: int64

In [274]:
# using a1
pd.DataFrame(a1).value_counts()[:20]

0                 
frank h. knight       113
george j. stigler      95
a. b. wolfe            92
paul a. samuelson      91
j. m. clark            88
daron acemoglu         85
f. w. taussig          84
william j. baumol      83
m. bronfenbrenner      81
jean tirole            77
joseph e. stiglitz     75
paul h. douglas        71
h. p. willis           70
wesley c. mitchell     67
milton friedman        66
franklin m. fisher     64
harry g. johnson       63
frank a. fetter        60
t. n. carver           60
chester w. wright      58
Name: count, dtype: int64

In [275]:
# using a2
pd.DataFrame(a2).value_counts()[:20]

0                
f. h. knight         133
m. bronfenbrenner    101
j. m. clark           97
p. a. samuelson       96
g. j. stigler         95
a. b. wolfe           94
w. j. baumol          90
f. w. taussig         86
j. e. stiglitz        86
d. acemoglu           85
j. tirole             77
w. c. mitchell        75
h. p. willis          74
p. h. douglas         73
f. m. fisher          66
m. friedman           66
a. p. lerner          65
t. n. carver          64
h. g. johnson         63
f. a. fetter          63
Name: count, dtype: int64

In [276]:
# using a3
pd.DataFrame(i1).value_counts()[:20]

0                
f. knight            133
j. clark             112
f. fetter            111
r. gordon            101
m. bronfenbrenner    101
j. stiglitz           98
p. samuelson          98
g. stigler            97
a. wolfe              94
w. baumol             92
f. taussig            86
d. acemoglu           85
w. mitchell           83
m. feldstein          78
j. tirole             77
h. willis             74
v. smith              74
p. douglas            74
j. robinson           73
c. wright             71
Name: count, dtype: int64

## Frequency of Authors scale of output for all time

In [277]:
# after initial processing
summary=[]
seq=[0,1,2,5,10,50,100,200,1000]
nms=pd.DataFrame(pd.DataFrame(all_authors).value_counts()).reset_index()
for i in range(len(seq)-1):
#     print("between "+str(seq[i])+" exclusive and "+str(seq[i+1])+ " inclusive")
#     print(nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0])
    summary.append({"occurences":str(seq[i])+" < x <= "+str(seq[i+1]), "number":nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0]})
    
pd.DataFrame(summary)

,occurences,number
0,0 < x <= 1,12663
1,1 < x <= 2,3596
2,2 < x <= 5,3452
3,5 < x <= 10,1613
4,10 < x <= 50,1177
5,50 < x <= 100,33
6,100 < x <= 200,1
7,200 < x <= 1000,0


In [278]:
# first initial + last name
summary=[]
seq=[0,1,2,5,10,50,100,200,1000]
nms=pd.DataFrame(pd.DataFrame(i1).value_counts()).reset_index()
for i in range(len(seq)-1):
#     print("between "+str(seq[i])+" exclusive and "+str(seq[i+1])+ " inclusive")
#     print(nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0])
    summary.append({"occurences":str(seq[i])+" < x <= "+str(seq[i+1]), "number":nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0]})
    
pd.DataFrame(summary)

,occurences,number
0,0 < x <= 1,8662
1,1 < x <= 2,2839
2,2 < x <= 5,3140
3,5 < x <= 10,1640
4,10 < x <= 50,1374
5,50 < x <= 100,44
6,100 < x <= 200,5
7,200 < x <= 1000,0


In [279]:
# Initials and last name
summary=[]
seq=[0,1,2,5,10,50,100,200,1000]
nms=pd.DataFrame(pd.DataFrame(a2).value_counts()).reset_index()
for i in range(len(seq)-1):
    print("between "+str(seq[i])+" exclusive and "+str(seq[i+1])+ " inclusive")
    print(nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0])
    summary.append({"occurences":str(seq[i])+" < x <= "+str(seq[i+1]), "number":nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0]})
    
pd.DataFrame(summary)

between 0 exclusive and 1 inclusive
10725
between 1 exclusive and 2 inclusive
3252
between 2 exclusive and 5 inclusive
3352
between 5 exclusive and 10 inclusive
1639
between 10 exclusive and 50 inclusive
1257
between 50 exclusive and 100 inclusive
39
between 100 exclusive and 200 inclusive
2
between 200 exclusive and 1000 inclusive
0


,occurences,number
0,0 < x <= 1,10725
1,1 < x <= 2,3252
2,2 < x <= 5,3352
3,5 < x <= 10,1639
4,10 < x <= 50,1257
5,50 < x <= 100,39
6,100 < x <= 200,2
7,200 < x <= 1000,0


In [280]:
# contracted middle names
summary=[]
seq=[0,1,2,5,10,50,100,200,1000]
nms=pd.DataFrame(pd.DataFrame(a1).value_counts()).reset_index()
for i in range(len(seq)-1):
#     print("between "+str(seq[i])+" exclusive and "+str(seq[i+1])+ " inclusive")
#     print(nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0])
    summary.append({"occurences":str(seq[i])+" < x <= "+str(seq[i+1]), "number":nms[(nms["count"]<=seq[i+1])&(nms["count"]>seq[i])].shape[0]})
    
pd.DataFrame(summary)

,occurences,number
0,0 < x <= 1,12289
1,1 < x <= 2,3536
2,2 < x <= 5,3420
3,5 < x <= 10,1620
4,10 < x <= 50,1193
5,50 < x <= 100,34
6,100 < x <= 200,1
7,200 < x <= 1000,0


## Resolving problem names for JPE

It is just read in above but this is how it should be generated if the file above is not applied to the variable j_data. They are printed to a json file with a unique timestamp please resolve each name, using the URLs below to check for the author name on the article page if the names were cutoff.

In [281]:
# stuff={}
# for i in j_data.index:
#     authors=j_data.loc[i,"author"]
#     if authors is not np.NaN:
#         sa=authors.lower()
#         if ("{" in sa) or ("\\" in sa):
#             print(j_data.loc[i,"author"])
#             print(j_data.loc[i, "URL"])
#             stuff[j_data.loc[i, "author"]]=j_data.loc[i, "author"]
#             print()

# with open("jpe_problem_names_"+str(time.time())+".json", "w") as outfile: 
#     json.dump(stuff, outfile, indent=4)

## Resolving special characters

Todo: optimize this code.
As above, add in the the unresolvable characters directly in the file and rename it to match the file name above.

In [282]:
# spec_char_set={}

# spec_chars=[]
# def process_string(str_in):
#     if str_in.isascii()==True:
#         return str_in
#     hold=""
#     for o in str_in:
#         if o.isascii()==False:
#             spec_chars.append(o)
#             print(str_in)

# for i in j_data.index:
#     authors=j_data.loc[i,"author"]
#     if authors is not np.NaN:
#         sa=authors.lower()
#         process_string(sa)
        
# u_spec_chars=list(set(spec_chars))
# u_spec_chars.sort()

# for o in u_spec_chars:
#     spec_char_set[o]=unicodedata.normalize("NFKD", o).encode('ascii', 'ignore').decode('utf-8')
#     if len(spec_char_set[o])==0:
#         print(o)
        
# print("end")
        
# with open("spec_char_replacement_"+str(time.time())+".json", "w") as outfile: 
#     json.dump(spec_char_set, outfile, indent=4)